In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import sqlite3
import requests
import json
from pandas import json_normalize
import pandas as pd

# Load env. variables from .env file
load_dotenv()
api_key = os.getenv("RAPIDAPI_KEY")
api_host = os.getenv("RAPIDAPI_HOST")
db_path = os.getenv("db_path")
json_path = os.getenv("json_path")

In [41]:
# Load JSON file from data/json_files/season
with open("C:/Users/Lavoro/Desktop/AIBetting/core/data/json_files/2020/players_stats.json") as json_file:
    data = json.load(json_file)
    print(data)

[{'get': 'fixtures/players', 'parameters': {'fixture': '147072'}, 'errors': [], 'results': 2, 'paging': {'current': 1, 'total': 1}, 'response': [{'team': {'id': 596, 'name': 'Zenit Saint Petersburg', 'logo': 'https://media.api-sports.io/football/teams/596.png', 'update': '2020-05-13T13:52:41+00:00'}, 'players': [{'player': {'id': 1197, 'name': 'Andrey Lunev', 'photo': 'https://media.api-sports.io/football/players/1197.png'}, 'statistics': [{'games': {'minutes': 90, 'number': 99, 'position': 'G', 'rating': '7.3', 'captain': False, 'substitute': False}, 'offsides': None, 'shots': {'total': 0, 'on': 0}, 'goals': {'total': None, 'conceded': 0, 'assists': None, 'saves': 2}, 'passes': {'total': 17, 'key': 0, 'accuracy': '94%'}, 'tackles': {'total': None, 'blocks': 0, 'interceptions': 0}, 'duels': {'total': None, 'won': None}, 'dribbles': {'attempts': 0, 'success': 0, 'past': None}, 'fouls': {'drawn': 0, 'committed': 0}, 'cards': {'yellow': 0, 'red': 0}, 'penalty': {'won': None, 'commited': N

In [3]:
# normalize json
df = pd.json_normalize(data, sep = "_")
# Drop useless columns
df = df.drop(columns=['get', 'errors', 'results', 'paging_current', 'paging_total'])
# Renale parameter_fixture in fixture_id and set as index
df = df.rename(columns={'parameters_fixture': 'fixture_id'})
df.head()


,response,fixture_id
0,"[{'team': {'id': 596, 'name': 'Zenit Saint Pet...",147072
1,"[{'team': {'id': 2012, 'name': 'PFC Sochi', 'l...",147077
2,"[{'team': {'id': 2012, 'name': 'PFC Sochi', 'l...",147166
3,"[{'team': {'id': 596, 'name': 'Zenit Saint Pet...",147176
4,"[{'team': {'id': 621, 'name': 'Krasnodar', 'lo...",147177


In [4]:
# Normalize response column
df_teams=pd.json_normalize(df['response'])
# concat fixture_id to df_teams
df_teams = pd.concat([df['fixture_id'], df_teams], axis=1)
df_teams.head(11)

,fixture_id,0,1
0,147072,"{'players': [{'player': {'id': 1197, 'name': '...","{'players': [{'player': {'id': 53697, 'name': ..."
1,147077,"{'players': [{'player': {'id': 40244, 'name': ...","{'players': [{'player': {'id': 53618, 'name': ..."
2,147166,"{'players': [{'player': {'id': 40244, 'name': ...","{'players': [{'player': {'id': 7771, 'name': '..."
3,147176,"{'players': [{'player': {'id': 1197, 'name': '...","{'players': [{'player': {'id': 464, 'name': 'G..."
4,147177,"{'players': [{'player': {'id': 2068, 'name': '...","{'players': [{'player': {'id': 53697, 'name': ..."
5,147178,"{'players': [{'player': {'id': 43365, 'name': ...","{'players': [{'player': {'id': 53725, 'name': ..."
6,147179,"{'players': [{'player': {'id': 821, 'name': 'I...","{'players': [{'player': {'id': 53618, 'name': ..."
7,147180,"{'players': [{'player': {'id': 2618, 'name': '...","{'players': [{'player': {'id': 1764, 'name': '..."
8,147181,"{'players': [{'player': {'id': 53536, 'name': ...","{'players': [{'player': {'id': 53515, 'name': ..."
9,147182,"{'players': [{'player': {'id': 40244, 'name': ...","{'players': [{'player': {'id': 53487, 'name': ..."


In [12]:
# Normalize home team data in response
df_home = pd.json_normalize(df_teams[0])
df_home = df_home.add_prefix('home_')
df_home = df_home.rename(columns={'home_team_id': 'team_id'})
df_home.columns = df_home.columns.str.replace(".", "_")
# add fixture_id to df_home
df_home = pd.concat([df_teams['fixture_id'], df_home], axis=1)
df_home.head()


,fixture_id,home_players,home_team_id,home_team_name,home_team_logo,home_team_update
0,147072,"[{'player': {'id': 1197, 'name': 'Andrey Lunev...",596.0,Zenit Saint Petersburg,https://media.api-sports.io/football/teams/596...,2020-05-13T13:52:41+00:00
1,147077,"[{'player': {'id': 40244, 'name': 'Soslan Dzha...",2012.0,PFC Sochi,https://media.api-sports.io/football/teams/201...,2020-05-13T13:52:45+00:00
2,147166,"[{'player': {'id': 40244, 'name': 'Soslan Dzha...",2012.0,PFC Sochi,https://media.api-sports.io/football/teams/201...,2020-05-13T13:54:06+00:00
3,147176,"[{'player': {'id': 1197, 'name': 'Andrey Lunev...",596.0,Zenit Saint Petersburg,https://media.api-sports.io/football/teams/596...,2020-05-13T13:54:15+00:00
4,147177,"[{'player': {'id': 2068, 'name': 'Matvey Safon...",621.0,Krasnodar,https://media.api-sports.io/football/teams/621...,2020-05-13T13:54:16+00:00


In [13]:
# Normalize away team data in response
df_away = pd.json_normalize(df_teams[1])
df_away = df_away.add_prefix('away_')
df_away = df_away.rename(columns={'away_team_id': 'team_id'})
df_away.columns = df_away.columns.str.replace(".", "_")
df_away.head()

,away_players,away_team_id,away_team_name,away_team_logo,away_team_update
0,"[{'player': {'id': 53697, 'name': 'Aleksandr B...",1078.0,FC UFA,https://media.api-sports.io/football/teams/107...,2020-05-13T13:52:41+00:00
1,"[{'player': {'id': 53618, 'name': 'Yaroslav Go...",1084.0,Ural,https://media.api-sports.io/football/teams/108...,2020-05-13T13:52:45+00:00
2,"[{'player': {'id': 7771, 'name': 'Andrei Klimo...",1080.0,Orenburg,https://media.api-sports.io/football/teams/108...,2020-05-13T13:54:06+00:00
3,"[{'player': {'id': 464, 'name': 'Guilherme Mar...",597.0,Lokomotiv Moscow,https://media.api-sports.io/football/teams/597...,2020-05-13T13:54:15+00:00
4,"[{'player': {'id': 53697, 'name': 'Aleksandr B...",1078.0,FC UFA,https://media.api-sports.io/football/teams/107...,2020-05-13T13:54:16+00:00


In [25]:
# merge home and away data
df_players = pd.concat([df_home, df_away], axis=1)
df_players.head()

,fixture_id,home_players,home_team_id,home_team_name,home_team_logo,home_team_update,away_players,away_team_id,away_team_name,away_team_logo,away_team_update
0,147072,"[{'player': {'id': 1197, 'name': 'Andrey Lunev...",596.0,Zenit Saint Petersburg,https://media.api-sports.io/football/teams/596...,2020-05-13T13:52:41+00:00,"[{'player': {'id': 53697, 'name': 'Aleksandr B...",1078.0,FC UFA,https://media.api-sports.io/football/teams/107...,2020-05-13T13:52:41+00:00
1,147077,"[{'player': {'id': 40244, 'name': 'Soslan Dzha...",2012.0,PFC Sochi,https://media.api-sports.io/football/teams/201...,2020-05-13T13:52:45+00:00,"[{'player': {'id': 53618, 'name': 'Yaroslav Go...",1084.0,Ural,https://media.api-sports.io/football/teams/108...,2020-05-13T13:52:45+00:00
2,147166,"[{'player': {'id': 40244, 'name': 'Soslan Dzha...",2012.0,PFC Sochi,https://media.api-sports.io/football/teams/201...,2020-05-13T13:54:06+00:00,"[{'player': {'id': 7771, 'name': 'Andrei Klimo...",1080.0,Orenburg,https://media.api-sports.io/football/teams/108...,2020-05-13T13:54:06+00:00
3,147176,"[{'player': {'id': 1197, 'name': 'Andrey Lunev...",596.0,Zenit Saint Petersburg,https://media.api-sports.io/football/teams/596...,2020-05-13T13:54:15+00:00,"[{'player': {'id': 464, 'name': 'Guilherme Mar...",597.0,Lokomotiv Moscow,https://media.api-sports.io/football/teams/597...,2020-05-13T13:54:15+00:00
4,147177,"[{'player': {'id': 2068, 'name': 'Matvey Safon...",621.0,Krasnodar,https://media.api-sports.io/football/teams/621...,2020-05-13T13:54:16+00:00,"[{'player': {'id': 53697, 'name': 'Aleksandr B...",1078.0,FC UFA,https://media.api-sports.io/football/teams/107...,2020-05-13T13:54:16+00:00


# da modificare da qui

In [26]:
# normalize home_players_info
df_home_players_info = pd.json_normalize(df_players['home_players'][0])
# Add prefix to columns
df_home_players_info = df_home_players_info.add_prefix('home_player_')
# Concat df_home players with df_players
df_home_players_info = pd.concat([df_players['fixture_id'], df_home_players_info], axis=1)

df_home_players_info.head()


,fixture_id,home_player_statistics,home_player_player.id,home_player_player.name,home_player_player.photo
0,147072,"[{'games': {'minutes': 90, 'number': 99, 'posi...",1197.0,Andrey Lunev,https://media.api-sports.io/football/players/1...
1,147077,"[{'games': {'minutes': 90, 'number': 15, 'posi...",2620.0,Vyacheslav Karavaev,https://media.api-sports.io/football/players/2...
2,147166,"[{'games': {'minutes': 90, 'number': 4, 'posit...",2443.0,Yordan Osorio,https://media.api-sports.io/football/players/2...
3,147176,"[{'games': {'minutes': 90, 'number': 44, 'posi...",1204.0,Yaroslav Rakitskiy,https://media.api-sports.io/football/players/1...
4,147177,"[{'games': {'minutes': 67, 'number': 3, 'posit...",24866.0,Douglas Santos,https://media.api-sports.io/football/players/2...


In [27]:
# normalize away_players_info
df_away_players_info = pd.json_normalize(df_players['away_players'][0])
# Add prefix to columns
df_away_players_info = df_away_players_info.add_prefix('away_player_')
# Concat df_home players with df_players
df_away_players_info = pd.concat([df_players['fixture_id'], df_away_players_info], axis=1)

df_away_players_info.head()

,fixture_id,away_player_statistics,away_player_player.id,away_player_player.name,away_player_player.photo
0,147072,"[{'games': {'minutes': 90, 'number': 31, 'posi...",53697.0,Aleksandr Belenov,https://media.api-sports.io/football/players/5...
1,147077,"[{'games': {'minutes': 90, 'number': 77, 'posi...",53710.0,Azer Aliev,https://media.api-sports.io/football/players/5...
2,147166,"[{'games': {'minutes': 90, 'number': 27, 'posi...",53704.0,Ionuț Nedelcearu,https://media.api-sports.io/football/players/5...
3,147176,"[{'games': {'minutes': 90, 'number': 15, 'posi...",53706.0,Aleksandr Putsko,https://media.api-sports.io/football/players/5...
4,147177,"[{'games': {'minutes': 90, 'number': 55, 'posi...",53708.0,Jemal Tabidze,https://media.api-sports.io/football/players/5...


In [28]:
# join home and away players on fixture_id
df_players_info = pd.merge(df_home_players_info, df_away_players_info, on='fixture_id')
df_players_info.head()

,fixture_id,home_player_statistics,home_player_player.id,home_player_player.name,home_player_player.photo,away_player_statistics,away_player_player.id,away_player_player.name,away_player_player.photo
0,147072,"[{'games': {'minutes': 90, 'number': 99, 'posi...",1197.0,Andrey Lunev,https://media.api-sports.io/football/players/1...,"[{'games': {'minutes': 90, 'number': 31, 'posi...",53697.0,Aleksandr Belenov,https://media.api-sports.io/football/players/5...
1,147077,"[{'games': {'minutes': 90, 'number': 15, 'posi...",2620.0,Vyacheslav Karavaev,https://media.api-sports.io/football/players/2...,"[{'games': {'minutes': 90, 'number': 77, 'posi...",53710.0,Azer Aliev,https://media.api-sports.io/football/players/5...
2,147166,"[{'games': {'minutes': 90, 'number': 4, 'posit...",2443.0,Yordan Osorio,https://media.api-sports.io/football/players/2...,"[{'games': {'minutes': 90, 'number': 27, 'posi...",53704.0,Ionuț Nedelcearu,https://media.api-sports.io/football/players/5...
3,147176,"[{'games': {'minutes': 90, 'number': 44, 'posi...",1204.0,Yaroslav Rakitskiy,https://media.api-sports.io/football/players/1...,"[{'games': {'minutes': 90, 'number': 15, 'posi...",53706.0,Aleksandr Putsko,https://media.api-sports.io/football/players/5...
4,147177,"[{'games': {'minutes': 67, 'number': 3, 'posit...",24866.0,Douglas Santos,https://media.api-sports.io/football/players/2...,"[{'games': {'minutes': 90, 'number': 55, 'posi...",53708.0,Jemal Tabidze,https://media.api-sports.io/football/players/5...
